In [56]:
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL
from spacy.pipeline import SpanCategorizer
from spacy.training import Example
from spacy.tokens import SpanGroup
import spacy
import json


In [57]:
train_data=json.load(open('./data/training/train_data.json'))
nlp = spacy.load("en_core_web_lg")

In [58]:
nlp=spacy.blank('en')

config = {
    "threshold": 0.5,
    "spans_key": "labeled_spans",
    "max_positive": None,
    "model": DEFAULT_SPANCAT_MODEL,
    "suggester": {"@misc": "spacy.ngram_suggester.v1", "sizes": [1, 2, 3]},
}

if 'spancat' not in nlp.pipe_names:
    spancat=nlp.create_pipe('spancat')
    nlp.add_pipe('spancat',config=config)


In [59]:

for _,annotations in train_data:
    for ent in annotations['entities']:
        spancat.add_label(ent[2])
other_pipes=[pipe for pipe in nlp.pipe_names if pipe !='spancat']


In [60]:
nlp.pipe_names
spancat.labels

('Companies worked at',
 'Skills',
 'Graduation Year',
 'College Name',
 'Degree',
 'Designation',
 'Email Address',
 'Location',
 'Name',
 'Years of Experience',
 'UNKNOWN')

In [ ]:
from numpy import random

with nlp.disable_pipes(*other_pipes):
    optimizer=spancat.create_optimizer()
    for batch in spacy.util.minibatch(train_data,size=2):
        random.shuffle(train_data)
        index=0
        doc=doc.spans(txt)
        
        for txt,annotation in batch:
            example=Example.from_dict(doc,annotation)
            try:
                losses=spancat.update([example],
                sgd=optimizer,
                drop=0.2,)
                print(losses)
            except Exception as e:
                print(e)